In [12]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

import pickle
import torch
import torch.nn as nn

In [4]:
dir00 = '/home/watson/Documents/CIFAR/cifar-10-python/cifar-10-batches-py/cifar-10-batches-py/small_alexnet'
sub_folder = 'corrupt_labels_100_12-03-2024_19-00-33'
file_weights = 'model_weights4950.pth'
file_results = 'file4999.pkl'

directory = dir00 + os.sep + sub_folder
weights_dir = directory + os.sep + file_weights
results_dir = directory + os.sep + file_results

In [13]:
class Conv(nn.Module):
    def __init__(self,input_channel,output_channel,kernel_size,stride,padding):
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(input_channel, output_channel, kernel_size=kernel_size, stride=stride, padding=padding)
        self.max = nn.MaxPool2d(3, stride=2,padding=0)
        self.localnorm = nn.LocalResponseNorm(5,alpha=1e-4,beta=0.75,k=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.max(x)
        x = self.localnorm(x)
        x = self.relu(x)
        return x
    
class fullconnect(nn.Module):
    def __init__(self,input_channel,output_channel):
        super(fullconnect, self).__init__()
        self.fc = nn.Linear(input_channel,output_channel)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        return x
    
class AlexnetSmall(nn.Module):
    def __init__(self,input_channel):
        super(AlexnetSmall, self).__init__()
        self.conv1 = Conv(input_channel,128,5,1,2)
        self.conv2 = Conv(128,256,5,1,2)
        self.maxpool = nn.MaxPool2d(3, stride=2)

        self.fc1 = fullconnect(1024,384)
        self.fc2 = fullconnect(384,192)
        self.fc3 = nn.Linear(192, 10)  # 10-way classification

    def forward(self, x):
        x = self.conv1(x)
        # print('conv1 done',x.shape)
        x = self.conv2(x)
        # print('conv2 done',x.shape)
        x = self.maxpool(x)
        # print('max pool is done',x.shape)
        x = torch.flatten(x,1)
        # print('x dimension after flattening',x.shape)
        x = self.fc1(x)
        # print('fc1 done')
        x = self.fc2(x)
        # print('fc2 done')
        x = self.fc3(x)
        # print('fc3 done')
        return x

In [5]:
with open(results_dir,'rb') as f:
    results = pickle.load(f)


In [ ]:
model = AlexnetSmall(3)
model.load_state_dict(torch.load(weights_dir,weights_only=True))

In [ ]:
norm = []
for param in model.parameters():
    
    print(np.sum(param))

In [ ]:
model.parameters